# 事前準備
## ライブラリのインストール

In [ ]:
from urllib.parse import urlparse

def classify_source(url):
    """
    分類リクエストを送信する

    Args:
        message (str): メッセージ
    """
    parsed_url = urlparse(url)
    if parsed_url.netloc == "brainpad.atlassian.net":
        return "confluence"
    elif parsed_url.netloc == "blog.brainpad.co.jp":
        return "platinum_data_blog"

In [ ]:
!python.exe -m pip install --upgrade pip
!pip install atlassian-python-api
!pip install langchain
!pip install typing_extensions==4.5.0
!pip install typing-inspect==0.8.0
!pip install beautifulsoup4
!pip install lxml
!pip install pytesseract Pillow
!pip install openai
!pip install tiktoken

In [ ]:
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from atlassian import Confluence

from langchain.document_loaders import ConfluenceLoader
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from openai import ChatCompletion

## 設定クラス

In [ ]:
class Conf:
    def __init__(self, user_name, confluence_api_key, openai_api_key):
        self.user_name = user_name
        self.confluence_api_key = confluence_api_key
        self.openai_api_key = openai_api_key
conf = Conf(
    user_name="",
    confluence_api_key="",
    openai_api_key="",
)
# CONFLUENCE_USERNAME
# CONFLUENCE_API_KEY


# 実装

## Confluence

In [ ]:
class ConfluencePage:
    def __init__(self, url):
        self.url = url

    @property
    def base_url(self):
        parsed_url = urlparse(self.url)
        return parsed_url.scheme + "://" + parsed_url.netloc

    @property
    def space_id(self):
        return self.url.split('/')[5]
    
    @property
    def page_id(self):
        return self.url.split('/')[7]
    
    def get_documents(self, user_name, api_key):
        loader = ConfluenceLoader(
            url=self.base_url,
            username=user_name,
            api_key=api_key
        )
        documents = loader.load(
            page_ids=[self.page_id],
            include_attachments=False,
            limit=10)
        return documents

    def get_html(self, user_name, api_key):
        confluence = Confluence(
            url=self.base_url,
            username=user_name,
            password=api_key
        )
        page_info = confluence.get_page_by_id(page_id=self.page_id, expand='body.storage')
        
        return page_info["body"]["storage"]["value"]

## LLM

In [ ]:
class DocumentPreprocessor:
    def __init__(self) -> None:
        pass

    def create_document(self, html,  url, chunk_size=1000):
        text = self.html_to_text(html)
        docs = self.text_to_documents(text=text, url=url, chunk_size=chunk_size)
        return docs

    def html_to_text(self, html):
        """
        HTMLをテキストに変換します
        """
        soup=BeautifulSoup(html,"html.parser")
        text=soup.get_text('\n')
        lines= [line.strip() for line in text.splitlines()]
        return "\n".join(lines)

    def text_to_documents(self, text, url, chunk_size):
        """
        ドキュメントをいくつかのチャンクに分割します。
        """
        text_splitter = CharacterTextSplitter(separator="\n", chunk_size=chunk_size, chunk_overlap=0)
        docs = text_splitter.create_documents([text], metadatas=[{"source": url}])
        return docs

In [ ]:
class QuestionGenerator:
    def __init__(self, llm):
        self.conf = conf
        self.llm = llm

    def get_topics(self, docs):
        # トピックの抽出
        query = """
        この文章のトピックを一言で表すと何と言えますか
        """
        chain = load_qa_with_sources_chain(self.llm, chain_type="map_reduce")
        topic_text = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
        return topic_text
    
    def generate_topic_issues(self, topic):
        # 論点の抽出
        template = """
        以下のトピックについて議論をする際に重要な論点を網羅的に羅列してください。
        {topic_text}
        """
        prompt = PromptTemplate(
            template=template,
            input_variables=["topic_text"]
        )
        prompt_text = prompt.format(topic_text=[topic])

        # 
        response = ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt_text}
            ]
        )

        #ChatGPTの回答を出力
        return response["choices"][0]["message"]["content"]
    
    def find_missing_issues(self, issues, docs):
        template = """
        以下の重要な論点のうち、ここまでの議論で取り上げられていないものはどれですか
        {issue_text}
        """
        prompt = PromptTemplate(
            template=template,
            input_variables=["issue_text"]
        )
        prompt_text = prompt.format(issue_text=issues)
        print(prompt_text)

        chain = load_qa_with_sources_chain(self.llm, chain_type="map_reduce")
        question_points = chain({"input_documents": docs, "question": prompt_text}, return_only_outputs=True)
        return question_points
    
    def generate_question(self, missing_issues):
        # プロンプトの生成
        template = """
        あなたは親切なアシスタントです。質問したい内容についてより深く知るための質問を5つ生成してください。
        質問は全て「大変興味深い発表ありがとうございます。素人質問で恐縮ですが、」に続く形で簡潔にしてください。

        # 質問したい内容
        {missing_issues}

        # フォーマット
        1. [question1]
        2. [question2]
        3. [question3]
        4. [question4]
        5. [question5]
        """
        prompt = PromptTemplate(
            template=template,
            input_variables=["missing_issues"]
        )
        prompt_text = prompt.format(
            missing_issues=missing_issues)

        # 質問の生成
        response = ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt_text}
            ]
        )

        #ChatGPTの回答を出力
        return response["choices"][0]["message"]["content"]

# 実行

## 前処理

In [ ]:
url = "https://****" # コンペ事業
# conflueceデータの読み込み
confulence_page = ConfluencePage(
    url=url,
)
html = confulence_page.get_html(
    user_name=conf.user_name,
    api_key=conf.confluence_api_key,
    )
html

In [ ]:
document_prep = DocumentPreprocessor()
docs = document_prep.create_document(html, url)
docs

## 質問の生成

In [ ]:
# モデルの読み込み
llm = ChatOpenAI(openai_api_key=conf.openai_api_key, model_name= "gpt-3.5-turbo", temperature=0.3)
qgen = QuestionGenerator(llm)
topic = qgen.get_topics(docs)
issues = qgen.generate_topic_issues(topic["output_text"])
missing_issues = qgen.find_missing_issues(issues, docs)
question = qgen.generate_question(missing_issues)
print("[topic]\n", topic)
print("[issues]\n", issues)
print("[missing_issues]\n", missing_issues["output_text"])
print("[question]\n", question)
question

## STEP1 内容を要約する

### STEP1 質問すべきポイントを指摘する

In [ ]:

template = """
以下の重要な論点のうち、ここまでの議論で取り上げられていないものはどれですか
{issue_text}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["issue_text"]
)
prompt_text = prompt.format(issue_text=[response["choices"][0]["message"]["content"]])
print(prompt_text)

chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_points = chain({"input_documents": docs, "question": prompt_text}, return_only_outputs=True)
question_points

- ブレインパッドの新規事業の内容を具体的に教えて。 output:ja
    - {'output_text': ' BrainPadの新規事業の内容は、三者間の協力によってメリットを得られること、プレゼンスを向上させること、競合他社との差別化を図ること、優秀な参加者への唾つけをすること、案件外組織との関係を獲得すること、受託分析に流せる可能性を持つこと、AS部の既存課題を改善するシナジーを生むことなどがあります。\nSOURCES: https://brainpad.atlassian.net/wiki/spaces/~yuta.yoshida/pages/3595404469/AS'}
- 資料の内容を踏まえて感想文を180文字程度で書いて
    - {'output_text': ' ブレインパッド社が開催するデータ分析コンペ「白金鉱業Cup #1」は、受託分析とは異なるプランでありながら、金銭価値だけでなくリード獲得やコネクション獲得、採用貢献などの多くの効果を期待できると私は考えます。参加企業をはじめ大学・行政・toC企業・ベンチャーなどが関わり、それぞれが持つメリットを生かし、多様な価値を創出していくことが可能であると'}
- ここまでの内容を踏まえて、重要な説明が不足しているポイントを2つ教えてください。
    - {'output_text': ' The two points that are missing important explanations are 1) the cost (how much people can be allocated) and 2) the opportunities beyond the monetary value from the competition itself, such as lead acquisition, connection acquisition, contribution to recruitment (cost reduction to recruiters), enhancement of brand image, partial improvement of organizational issues, differentiation from other companies, etc.\nSOURCES: https://brainpad.atlassian.net/wiki/spaces/~yuta.yoshida/pages/3595404469/AS'}
- ここまでの内容を踏まえて、重要であるが説明が不足しているポイントは何がありますか？
    - Based on the given portion of the document, it is difficult to determine a specific point that is important but lacking in explanation. However, potential areas where more explanation may be needed include the specific goals and objectives of the proposed ML competitions and data hackathons, the potential benefits and drawbacks of the proposed initiatives, the resources and support needed to implement the initiatives, the potential impact on the AS department's reputation and relationships, the potential risks and challenges associated with the initiatives, and the potential role of the AS department in promoting and facilitating the initiatives

### STEP2 質問文の形に整形する

In [ ]:
template = """
あなたは親切なアシスタントです。質問したい内容についてより深く知るために、ここまでの議論の要約を踏まえて「大変興味深い発表ありがとうございます。素人質問で恐縮ですが、」に続く形で質問をしてください。

# 質問したい内容
{question_points}

# ここまでの議論の要約
{summary}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["question_points", "summary"]
)
prompt_text = prompt.format(
    question_points=[question_points["output_text"]],
    summary=[summary])
print(prompt_text)

response = ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": prompt_text}
    ]
)

#ChatGPTの回答を出力
print(response["choices"][0]["message"]["content"])

あなたは親切なアシスタントです。以下の内容についてもう少し深く知るために「大変興味深い発表ありがとうございます。素人質問で恐縮ですが、」に続く形で質問をしてください。複数のポイントがある場合、重要な一つに絞って質問するようにしてください。
["Based on the given portion of the document, it is difficult to determine a specific point that is important but lacking in explanation. However, potential areas where more explanation may be needed include the specific goals and objectives of the proposed ML competitions and data hackathons, the potential benefits and drawbacks of the proposed initiatives, the resources and support needed to implement the initiatives, the potential impact on the AS department's reputation and relationships, the potential risks and challenges associated with the initiatives, and the potential role of the AS department in promoting and facilitating the initiatives. \nSOURCES: https://brainpad.atlassian.net/wiki/spaces/~yuta.yoshida/pages/3595404469/AS"]

大変興味深い発表ありがとうございます。素人質問で恐縮ですが、提案されたMLの競技会やデータハッカソンの具体的な目標や目的、提案されたイニシアチブの潜在的な利点や欠点、イニシアチブを実施するために必要なリソースやサポート、AS部門の評判や関係に与える潜在的な影響、イニシアチブに関連する潜在的なリスクや課題、そしてAS部門のイニシアチブを促進・支援する上での潜在的な役割について、もう少し詳しく教えていただけますか？